In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt

from plot_utils import df_heatmap
from red_likelihood import decomposeCArray, degVis, makeCArray, red_ant_sep
from red_utils import find_nearest, find_deg_df, find_rel_df, \
match_lst

In [ ]:
%matplotlib inline

In [ ]:
jd_time = 2458098.43869
jd_comp = 2458099
pol = 'ee'
dist = 'gaussian'
dir_path = '/Users/matyasmolnar/Downloads/HERA_Data/robust_cal/simpleredcal'
coords = 'cartesian'

In [ ]:
plot_figs = False
if plot_figs:
    import matplotlib as mpl
    mpl.rcParams['figure.dpi'] = 300

## Loading 1st relatively calibrated dataframe

In [ ]:
with open('rel_df.{}.{}.md.pkl'.format(jd_time, pol), 'rb') as f:
    md = pickle.load(f)
    
indices = ['freq', 'time_int']
resid_cols = ['residual', 'norm_residual']
vis_list = list(map(str, np.arange(md['no_unq_bls']*2).tolist()))
cvis_list = ['C' + vis_id for vis_id in list(map(str, np.arange(md['no_unq_bls']).tolist()))]
gain_list = list(map(str, np.arange(md['no_unq_bls']*2, (md['no_unq_bls'] + md['no_ants'])*2 ).tolist()))

rel_df_path = find_rel_df(jd_time, pol, dist, dir_path)
rel_df = pd.read_pickle(rel_df_path)
rel_df.drop(columns=resid_cols, inplace=True)

Nfreqs = rel_df.index.get_level_values('freq').unique().size
Ntints = rel_df.index.get_level_values('time_int').unique().size

## Loading 2nd relatively calibrated dataframe

Due to an offset in LAST, two relatively calibrated dataframes must be merged, with the appropriate cuts in LAST to align the merged dataframe with the 1st one

In [ ]:
# find dataset from specified JD that contains visibilities at the same LAST
jd_time2 = match_lst(jd_time, jd_comp)
rel_df_path2 = find_rel_df(jd_time2, pol, dist, dir_path)

# aligning datasets in LAST
last_df = pd.read_pickle('jd_lst_map_idr2.pkl')
last1 = last_df[last_df['JD_time'] == jd_time]['LASTs'].values[0]
last2 = last_df[last_df['JD_time'] == jd_time2]['LASTs'].values[0]
_, offset = find_nearest(last2, last1[0])

rel_df2 = pd.read_pickle(rel_df_path2)
rel_df2 = rel_df2[rel_df2.index.get_level_values('time_int') >= offset]
# shifting tints to align with those from jd_time
rel_df2.reset_index(inplace=True)
rel_df2['time_int'] = np.tile(np.arange(Ntints - offset), Nfreqs)
rel_df2.set_index(indices, inplace=True)

next_row = np.where(last_df['JD_time'] == jd_time2)[0][0] + 1
rel_df_path3 = find_rel_df(last_df.iloc[next_row]['JD_time'], pol, \
                           dist, dir_path)
rel_df3 = pd.read_pickle(rel_df_path3)
rel_df3 = rel_df3[rel_df3.index.get_level_values('time_int') < offset]
# shifting tints to align with those from jd_time
rel_df3.reset_index(inplace=True)
rel_df3['time_int'] = np.tile(np.arange(Ntints - offset, Ntints), Nfreqs)
rel_df3.set_index(indices, inplace=True)

# combined results dataframes that is now alinged in LAST by row number
# with rel_df:
rel_df_c = pd.concat([rel_df2, rel_df3])
rel_df_c.sort_index(inplace=True)
rel_df_c.drop(columns=resid_cols + gain_list, inplace=True)

## Degenerate transformation of the 1st dataframe

In [ ]:
min_list = ['success', 'status', 'message', 'fun', 'nit']
rel_df_d = rel_df[min_list].copy()
rel_df_d = rel_df_d.reindex(columns=rel_df_d.columns.values.tolist() + vis_list)
rel_df_d.sample(5).sort_index()

In [ ]:
deg_df_path = find_deg_df(jd_time, pol, 'jd.{}'.format(jd_comp), dist, dir_path)
deg_df = pd.read_pickle(deg_df_path)

In [ ]:
deg_df_d = deg_df[['0', '1', '2']].copy().reset_index()
deg_df_d.rename(columns={'time_int1': 'time_int', '0': 'amp', '1': 'tilt_x', '2':'tilt_y'}, inplace=True)
deg_df_d.set_index(indices, inplace=True)
deg_df_d.sort_index(inplace=True)
rel_df.drop(columns=gain_list, inplace=True)
rel_df = rel_df.join(deg_df_d)
rel_df.sample(5).sort_index()

In [ ]:
ant_sep = red_ant_sep(md['redg'], md['antpos'])
vis_list = list(map(str, np.arange(md['no_unq_bls']*2).tolist()))
rel_df_d[vis_list] = rel_df.apply(lambda row: pd.Series(decomposeCArray(degVis(ant_sep, \
                     makeCArray(row[len(min_list):len(min_list) + md['no_unq_bls']*2].values.astype(float)), 
                     *row[-3:].values.astype(float)))), axis=1)

In [ ]:
rel_df_d.sample(5).sort_index()

## Combining degenerately consistent dataframes

In [ ]:
# merging dataframes
rel_df_d['JD'] = int(jd_time)
rel_df_c['JD'] = int(jd_comp)

rel_df_t = pd.concat([rel_df_d, rel_df_c])

rel_df_t.reset_index(inplace=True)
rel_df_t.set_index(['freq', 'time_int', 'JD'], inplace=True)
rel_df_t.sort_index(inplace=True)

In [ ]:
rel_df_t.sample(5).sort_index()

## Statistics on combined dataframe

In [ ]:
rel_df_t[vis_list].groupby(level=['freq', 'time_int']).mean()

In [ ]:
rel_df_t[vis_list].groupby(level=['freq', 'time_int']).std()

## Plots

In [ ]:
time_integration = 20
a = rel_df_t.apply(lambda row: pd.Series(makeCArray(row[vis_list].values.astype(float))), axis=1).abs()

In [ ]:
b = a.xs(time_integration, level='time_int', drop_level=True).groupby(level=['freq']).mean()

In [ ]:
piv = pd.pivot_table(b, columns='freq')
vmax = np.nanpercentile(piv.values, 95)
vmin = 0
df_heatmap(piv, xbase=50, ybase=5, vmax=vmax, vmin=vmin, \
           title='Visibility amplitudes for time_integration {}'.format(time_integration), \
           xlabel='Frequency', \
           ylabel='Redundant Baseline Group')